In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /opt/anaconda3/envs/PythonData/lib/python3.6/site-packages (0.0)


In [2]:
# Install joblib - this will be used to save your model
# Restart your kernel after installing 
!pip install joblib

In [1]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [61]:
df = pd.read_csv("starter_code/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [62]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [63]:
# Set features. This will also be used as your x values.
df2 = df.drop("koi_disposition", axis = 1)
selected_features = df2[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag']]
selected_features

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,...,-152,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
6987,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,...,-166,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
6988,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,-0.007690,...,-220,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
6989,0,0,1,0,0.681402,2.430000e-06,-2.430000e-06,132.181750,0.002850,-0.002850,...,-236,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [74]:
df2

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,...,-152,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
6987,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,...,-166,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
6988,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,-0.007690,...,-220,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
6989,0,0,1,0,0.681402,2.430000e-06,-2.430000e-06,132.181750,0.002850,-0.002850,...,-236,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [64]:
# Set X and y
X = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_slogg', 'koi_srad']]
y = df['koi_disposition']
print(X.shape, y.shape)

(6991, 10) (6991,)


In [65]:
# Model Type: Random Forest
from sklearn.preprocessing import LabelEncoder
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [66]:
for label, original_class in zip(encoded_y, y):
    print('Original: ' + str(original_class))
    print('Encoded: ' + str(label))
    print('----------------')

Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Origi

Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
--------------

Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: 

Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Origi

----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------

...

Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
-------

----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Origin

----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
-------------

Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CONFIRMED
Encoded: 1
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Origi

Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: CANDIDATE
Encoded: 0
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
----------------
Original: FALSE POSITIVE
Encoded: 2
------------

In [67]:
from tensorflow.keras.utils import to_categorical
one_hot_y = to_categorical(encoded_y)
one_hot_y

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [77]:
# Create a train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, random_state=42, test_size=0.3)

In [78]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
4954,0,0,0,0,97.942437,0.001183,-0.001183,157.97325,0.00888,-0.00888,...,-198,3.806,0.445,-0.194,2.284,0.726,-1.090,288.53769,51.280384,11.366
4235,0,0,1,1,16.609490,0.000141,-0.000141,147.61311,0.00715,-0.00715,...,-210,4.395,0.105,-0.195,1.050,0.323,-0.162,300.43979,43.930729,14.926
848,0,0,0,0,32.275385,0.000541,-0.000541,149.04600,0.01400,-0.01400,...,-103,4.123,0.182,-0.098,1.483,0.230,-0.316,298.29495,48.775459,14.167
2874,0,1,1,1,0.787120,0.000002,-0.000002,131.73939,0.00190,-0.00190,...,-201,4.481,0.150,-0.100,0.787,0.093,-0.103,295.28110,40.132858,14.964
3016,0,0,0,0,6.963527,0.000039,-0.000039,133.01717,0.00435,-0.00435,...,-201,3.978,0.458,-0.153,1.686,0.413,-0.709,296.10406,42.342590,13.128


In [79]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)
X_scaler

MinMaxScaler()

In [80]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=600)
model.fit(scaled_X_train, y_train)
model.score(scaled_X_test, y_test)

0.877025738798856

In [81]:
print(f"Training Data Score: {model.score(scaled_X_train, y_train)}")
print(f"Testing Data Score: {model.score(scaled_X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.877025738798856


In [82]:
sorted(zip(model.feature_importances_, df.columns), reverse=True)

[(0.10668435065034257, 'koi_fpflag_ss'),
 (0.09848596051590398, 'koi_disposition'),
 (0.05996103577289038, 'koi_fpflag_nt'),
 (0.054187462088362774, 'koi_insol_err2'),
 (0.047987954615064096, 'koi_depth_err2'),
 (0.03665589364214901, 'koi_fpflag_co'),
 (0.035253290002911664, 'koi_steff'),
 (0.03422011363155107, 'koi_duration'),
 (0.03264749898497555, 'koi_prad'),
 (0.032115622159966474, 'koi_duration_err1'),
 (0.03101160993466772, 'koi_prad_err1'),
 (0.028358681752190893, 'koi_steff_err1'),
 (0.02492654106510108, 'koi_time0bk_err1'),
 (0.023783293524184193, 'koi_fpflag_ec'),
 (0.023694769228367805, 'koi_time0bk'),
 (0.022668281973323788, 'koi_duration_err2'),
 (0.021084450755500556, 'koi_impact_err2'),
 (0.01939168397767158, 'koi_period_err1'),
 (0.01861039551748669, 'koi_time0bk_err2'),
 (0.017514264654850165, 'koi_insol'),
 (0.017045297491985956, 'koi_period'),
 (0.016499360142938454, 'koi_prad_err2'),
 (0.014063132102931808, 'koi_depth_err1'),
 (0.013774771858240413, 'koi_depth'),
 

In [76]:
# test all features from dataset
X = df2[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag']]
# result: Training Data Score: 1.0, Testing Data Score: 0.8732125834127741

# test features with an importance of > 0.03

X2 = df2[['koi_fpflag_ss', 'koi_fpflag_nt', 
          'koi_insol_err2', 'koi_depth_err2', 'koi_duration_err1', 
          'koi_fpflag_co', 'koi_duration', 'koi_prad_err1', 'koi_steff']]
# result: Training Data Score: 1.0, Testing Data Score: 0.8760724499523356


In [57]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [200, 700],
              'max_features': ['auto', 'sqrt', 'log2']
}
grid = GridSearchCV(estimator = model, param_grid=param_grid, cv=5)

In [59]:
# Train the model with GridSearch
grid.fit(scaled_X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=600),
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 700]})

In [60]:
print(grid.best_params_)
print(grid.best_score_)

{'max_features': 'auto', 'n_estimators': 700}
0.8794145355115921


In [83]:
import joblib
filename = 'model_2.sav'
joblib.dump(model, "Model_2.sav")

['Model_2.sav']